# DC DATA - Collection / Cleaning / Preprocessing

## Imports

In [5]:
import pandas as pd
import numpy as np
import pandas as pd
import os
from sqlalchemy import create_engine
from DealMatch.database_connector import db_connector
import os
from dotenv import load_dotenv
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'DealMatch'

In [ ]:
pd.set_option('display.max_columns', 500)

In [ ]:
dbConnection = db_connector()

## DC Database queries

In [ ]:
query_investor_locations = """
        SELECT * 
        FROM companies c 
        LEFT JOIN invest_profiles ip ON ip.company_id = c.id
        LEFT JOIN invest_profile_locations ipl ON ipl.invest_profile_id = ip.id 
        LEFT JOIN invest_profile_countries ipc ON ipc.invest_profile_location_id = ipl.id 
        LEFT JOIN countries c2 ON c2.id = ipc.country_id 
        WHERE c.company_category_id = 1; 
"""

In [ ]:
query_investor_deals = """
        SELECT * 
        FROM companies c 
        LEFT JOIN deals d ON d.company_id = c.id 
        LEFT JOIN deal_stages ds ON ds.id = d.deal_stage_id 
        LEFT JOIN deal_types dt ON dt.id = d.deal_type_id 
        LEFT JOIN deal_lost_reasons dlr ON dlr.id = d.lost_reason_id 
        LEFT JOIN countries c2 ON d.target_country_id = c2.id 
        LEFT JOIN regions r ON d.target_region_id = r.id 
        JOIN deal_sectors ds2 ON ds2.deal_id = d.id 
        JOIN sectors s ON s.id = ds2.sector_id 
        JOIN deal_subsectors ds3 ON ds2.id = ds3.deal_sector_id 
        JOIN subsectors s2 ON s2.id = ds3.subsector_id 
        WHERE c.company_category_id = 1;
"""

In [ ]:
query_invest_profile_keywords = """
        SELECT * 
        FROM companies c 
        LEFT JOIN invest_profiles ip ON ip.company_id = c.id
        LEFT JOIN invest_profile_keywords ipk ON ipk.invest_profile_id = ip.id 
        LEFT JOIN keywords k ON k.id = ipk.keyword_id 
        WHERE c.company_category_id = 1;
"""

In [ ]:
query_invest_profile_sectors_subsectors = """
        SELECT * 
        FROM companies c 
        LEFT JOIN invest_profiles ip ON ip.company_id = c.id
        LEFT JOIN invest_profile_sectors ips ON ips.invest_profile_id = ip.id 
        LEFT JOIN sectors s ON s.id = ips.sector_id 
        LEFT JOIN invest_profile_subsectors ips2 ON ips2.invest_profile_sector_id = ips.id 
        LEFT JOIN subsectors s2 ON s2.id = ips2.subsector_id 
        WHERE c.company_category_id = 1;
"""

In [ ]:
query_invest_profile_transaction = """
        SELECT * 
        FROM companies c 
        LEFT JOIN invest_profiles ip ON ip.company_id = c.id
        LEFT JOIN invest_profile_tx_types iptt ON iptt.invest_profile_id = ip.id 
        LEFT JOIN tx_types tt ON tt.id = iptt.tx_type_id 
        LEFT JOIN invest_profile_stake_types ipst ON ipst.invest_profile_id = ip.id 
        LEFT JOIN stake_types st ON st.id = ipst.stake_type_id 
        WHERE c.company_category_id = 1;
"""

## RAW Dataframes

In [ ]:
investor_locations = pd.read_sql(query_investor_locations, dbConnection);
investor_locations.head()

In [ ]:
investor_locations.to_excel('investor_locations.xlsx', encoding="UTF-8")

In [ ]:
investor_deals = pd.read_sql(query_investor_deals, dbConnection);
investor_deals.head()

In [ ]:
investor_deals.to_excel('investor_deals.xlsx', encoding="UTF-8")

In [ ]:
invest_profile_keywords = pd.read_sql(query_invest_profile_keywords, dbConnection);
invest_profile_keywords.head()

In [ ]:
invest_profile_keywords.to_excel('invest_profile_keywords.xlsx', encoding="UTF-8")

In [ ]:
invest_profile_sectors_subsectors = pd.read_sql(query_invest_profile_sectors_subsectors, dbConnection);
invest_profile_sectors_subsectors.head()

In [ ]:
invest_profile_sectors_subsectors.to_excel('invest_profile_sectors_subsectors.xlsx', encoding="UTF-8")

In [ ]:
invest_profile_transaction = pd.read_sql(query_invest_profile_transaction, dbConnection);
invest_profile_transaction.head()

In [ ]:
invest_profile_transaction.to_excel('invest_profile_transaction.xlsx', encoding="UTF-8")

### Dataframe Shapes

In [ ]:
print(f"Invest-profile preferred locations df: {investor_locations.shape}")

In [ ]:
print(f"Investor Deals df: {investor_deals.shape}")

In [ ]:
print(f"Invest-profile preferred keywords df: {invest_profile_keywords.shape}")

In [ ]:
print(f"Invest-profile preferred sectors / subsectors df: {invest_profile_sectors_subsectors.shape}")

In [ ]:
print(f"Invest-profile preferred transaction & stake types df: {invest_profile_transaction.shape}")

## Investor Sector cleaning

In [ ]:
invest_profile_sectors_subsectors = pd.read_excel('invest_profile_sectors_subsectors.xlsx')

In [ ]:
invest_profile_sectors_subsectors.head()

In [ ]:
invest_profile_sectors_subsectors.dropna(axis=1, how="all", inplace=True)

In [ ]:
sectors_encoding = invest_profile_sectors_subsectors.copy()

In [ ]:
sectors_encoding.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
sectors_encoding.columns

In [ ]:
sectors_encoding = sectors_encoding[['investor_id', 'name', 'sector_id', 'is_excluded', 'name_en']]

In [ ]:
sectors_encoding.head()

In [ ]:
sectors_encoding.is_excluded.value_counts()

In [ ]:
sectors_encoding_no_exclude = sectors_encoding[sectors_encoding['is_excluded'] == 0.0]

In [ ]:
sectors_encoding_no_exclude.is_excluded.value_counts()

In [ ]:
sectors_encoding_no_exclude

In [ ]:
sectors_encoding_no_exclude_clean = sectors_encoding_no_exclude[['investor_id', 'name', 'name_en']]

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(sparse = False).fit(sectors_encoding_no_exclude_clean[['name_en']])

new_columns = list(one_hot_encoder.categories_[0])

sectors_encoding_no_exclude_clean[new_columns] = one_hot_encoder.transform(sectors_encoding_no_exclude_clean[['name_en']])

sectors_encoding_no_exclude_clean.head()


In [ ]:
sectors_encoding_no_exclude_clean.drop(columns=['name_en'], inplace=True)

In [ ]:
sectors_encoding_no_exclude_clean = sectors_encoding_no_exclude_clean.groupby('investor_id').max().reset_index()

In [ ]:
sectors_encoding_no_exclude_clean.name.nunique()

In [ ]:
sectors_encoding_no_exclude_clean.name = sectors_encoding_no_exclude_clean.name.apply(lambda x: x.strip())

In [ ]:
sectors_encoding_no_exclude_clean.drop_duplicates('name', keep="last", inplace=True)

In [ ]:
sectors_encoding_no_exclude_clean.head()

In [ ]:
sectors_encoding_no_exclude_clean.shape

In [ ]:
cols_to_sum = ['Agriculture', 'Automotive',
       'Biotechnology & Life Sciences', 'Chemicals',
       'Computer Hardware & Equipment', 'Construction',
       'Consumer Goods & Apparel', 'Defense', 'Electronics', 'Energy',
       'Financial Services', 'Food & Beverages', 'Food & Staples Retailing',
       'Government', 'Health Care Equipment & Services', 'IT services',
       'Industrial automation', 'Industrial products and services',
       'Insurance', 'Internet/ecommerce', 'Leisure & consumer services',
       'Manufacturing (other)', 'Media', 'Mining', 'Pharmaceuticals',
       'Professional Services (B2B)', 'Real Estate', 'Retailing',
       'Semiconductors & Semiconductor Equipment', 'Software & Services',
       'Telecommunication Hardware', 'Telecommunication Services',
       'Transportation', 'Utilities']

In [ ]:
sectors_encoding_no_exclude_clean['sector_count'] = sectors_encoding_no_exclude_clean[cols_to_sum].sum(axis=1)

In [ ]:
sectors_encoding_no_exclude_clean.sector_count.mean()

#### final cleaned df to merge = sectors_encoding_no_exclude_clean

## Transaction & Stake Types cleaning

In [ ]:
invest_profile_transaction = pd.read_excel('invest_profile_transaction.xlsx')

In [ ]:
invest_profile_transaction.head()

In [ ]:
invest_profile_transaction.dropna(axis=1, how="all", inplace=True)

In [ ]:
invest_profile_transaction.columns

In [ ]:
invest_profile_transaction_to_clean = invest_profile_transaction[['investor_id', 'name', 'fees_rate',
       'fees_cap', 'fees_floor', 'fees_fa', 'fees_info', 'inroad',
       'exit_oriented', 'management_takeover', 'seriousness', 'agnostic',
       'margin', 'tx_type_id', 'is_excluded', 'name.2', 'stake_type_id', 'stake_id', 'name.3']]

In [ ]:
invest_profile_transaction_to_clean.head()

In [ ]:
invest_profile_transaction_to_clean.fees_rate.replace(2,0.02, inplace=True)

In [ ]:
invest_profile_transaction_to_clean.fees_rate.replace(1,0.01, inplace=True)

In [ ]:
invest_profile_transaction_to_clean.fees_rate.replace(3,0.03, inplace=True)

In [ ]:
invest_profile_transaction_to_clean.fees_rate.replace(3,0.03, inplace=True)

In [ ]:
invest_profile_transaction_to_clean.fees_rate.replace(np.nan, 0, inplace=True)

In [ ]:
invest_profile_transaction_to_clean.fees_rate.value_counts()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(sparse = False).fit(invest_profile_transaction_to_clean[['fees_rate']])

new_columns = list(one_hot_encoder.categories_[0])

invest_profile_transaction_to_clean[new_columns] = one_hot_encoder.transform(invest_profile_transaction_to_clean[['fees_rate']])

invest_profile_transaction_to_clean.head()


In [ ]:
invest_profile_transaction_to_clean.drop(columns=['fees_rate', 'fees_cap', 'fees_floor', 'fees_fa', 'fees_info'], inplace=True)

In [ ]:
invest_profile_transaction_to_clean.head()

In [ ]:
invest_profile_transaction_to_clean.margin.replace(np.nan, "keine_marge", inplace=True)

In [ ]:
invest_profile_transaction_to_clean.margin.replace(0, 'keine_marge', inplace=True)
invest_profile_transaction_to_clean.margin.replace(1, 'undermanaged', inplace=True)
invest_profile_transaction_to_clean.margin.replace(2, 'mind_5', inplace=True)
invest_profile_transaction_to_clean.margin.replace(3, 'mind_10', inplace=True)
invest_profile_transaction_to_clean.margin.replace(4, 'mind_15', inplace=True)
invest_profile_transaction_to_clean.margin.replace(5, 'mind_20', inplace=True)

In [ ]:
one_hot_encoder = OneHotEncoder(sparse = False).fit(invest_profile_transaction_to_clean[['margin']])

new_columns = list(one_hot_encoder.categories_[0])

invest_profile_transaction_to_clean[new_columns] = one_hot_encoder.transform(invest_profile_transaction_to_clean[['margin']])

invest_profile_transaction_to_clean.head()

In [ ]:
invest_profile_transaction_to_clean.is_excluded.value_counts()

In [ ]:
invest_profile_transaction_no_exclude = invest_profile_transaction_to_clean[invest_profile_transaction_to_clean.is_excluded == 0]

In [ ]:
invest_profile_transaction_no_exclude.is_excluded.value_counts()

In [ ]:
invest_profile_transaction_no_exclude.drop(columns=['is_excluded'], inplace=True)

In [ ]:
invest_profile_transaction_no_exclude.drop(columns=['seriousness'], inplace=True)

In [ ]:
invest_profile_transaction_no_exclude.drop(columns=['margin'], inplace=True)

In [ ]:
invest_profile_transaction_no_exclude.head()

In [ ]:
invest_profile_transaction_no_exclude['name.2'] = invest_profile_transaction_no_exclude['name.2'].replace(np.nan, "no_info")

In [ ]:
invest_profile_transaction_no_exclude['name.2'].isna().sum()

In [ ]:
one_hot_encoder = OneHotEncoder(sparse = False).fit(invest_profile_transaction_no_exclude[['name.2']])

new_columns = list(one_hot_encoder.categories_[0])

invest_profile_transaction_no_exclude[new_columns] = one_hot_encoder.transform(invest_profile_transaction_no_exclude[['name.2']])

invest_profile_transaction_no_exclude.head()

In [ ]:
invest_profile_transaction_no_exclude.drop(columns=['name.2'], inplace=True)

In [ ]:
invest_profile_transaction_no_exclude['name.3'] = invest_profile_transaction_no_exclude['name.3'].replace(np.nan, 'no_stake_info')

In [ ]:
one_hot_encoder = OneHotEncoder(sparse = False).fit(invest_profile_transaction_no_exclude[['name.3']])

new_columns = list(one_hot_encoder.categories_[0])

invest_profile_transaction_no_exclude[new_columns] = one_hot_encoder.transform(invest_profile_transaction_no_exclude[['name.3']])

invest_profile_transaction_no_exclude.head()

In [ ]:
invest_profile_transaction_no_exclude.drop(columns=['name.3', 'tx_type_id', 'stake_type_id', 'stake_id', 'name.3'], inplace=True)

### FOR GAUTIER: Please check INROAD feature since this needs to be adjusted and changed according to the number of matches

In [ ]:
invest_profile_transaction_no_exclude.head()

In [ ]:
invest_profile_transaction_no_exclude.drop(columns=['inroad'], inplace=True)

In [ ]:
invest_profile_transaction_no_exclude.rename(columns={0.0: 'no_fee', 0.01: 'pays_1', 0.02: 'pays_2', 0.03: 'pays_3', 0.1:'fee_sharing_0.1', 0.25:'fee_sharing'}, inplace=True)


In [ ]:
invest_profile_transaction_no_exclude.rename(columns={'fee_sharing_0.1': 'fee_sharing_10', 'fee_sharing':'fee_sharing_25'}, inplace=True)


In [ ]:
invest_profile_transaction_no_exclude.head()

In [ ]:
invest_profile_transaction_no_exclude_test =  invest_profile_transaction_no_exclude.copy()

In [ ]:
invest_profile_transaction_no_exclude_final = invest_profile_transaction_no_exclude.copy()

In [ ]:
invest_profile_transaction_no_exclude_final = invest_profile_transaction_no_exclude_final.groupby('investor_id').sum()

In [ ]:
invest_profile_transaction_no_exclude_final[invest_profile_transaction_no_exclude_final > 1] = 1

In [ ]:
invest_profile_transaction_no_exclude_final.reset_index(inplace=True)

In [ ]:
invest_profile_transaction_no_exclude_final.head()

In [ ]:
invest_profile_transaction_no_exclude_final.investor_id.nunique()

#### final df to merge = invest_profile_transaction_no_exclude_final

## Investor Financials Cleaning

In [ ]:
invest_profile_financials = pd.read_excel('invest_profile_transaction.xlsx')
invest_profile_financials.head()

In [ ]:
invest_profile_financials.columns

In [ ]:
invest_profile_financials = invest_profile_financials[['investor_id', 'name', 'max_revenue', 'min_revenue',
       'max_ebitda', 'min_ebitda', 'max_equity', 'min_equity', 'name.2']]

In [ ]:
invest_profile_financials.head()

In [ ]:
test = invest_profile_financials.copy()

In [ ]:
test.head()

In [ ]:
res = test.groupby('investor_id',as_index=False)['name.2'].agg(list)

In [ ]:
test = pd.merge(test, res, left_on="investor_id", right_on="investor_id")

In [ ]:
test.drop(columns=['name.2_x'], inplace=True)

In [ ]:
test

In [ ]:
invest_profile_financials.groupby('name')['name.2'].apply(list).reset_index(name='transaction_types')

In [ ]:
invest_profile_financials.drop_duplicates(inplace=True)

In [ ]:
invest_profile_financials.shape

In [ ]:
invest_profile_financials.head()

In [ ]:
invest_profile_financials.to_excel('invest_profile_financials.xlsx', encoding="UTF-8")

In [ ]:
invest_profile_financials.columns

In [ ]:
iter_cols = ['max_revenue', 'min_revenue', 'max_ebitda',
       'min_ebitda', 'max_equity', 'min_equity']

In [ ]:
for col in iter_cols:
    g = sns.histplot(invest_profile_financials[col], bins=500)
    #g.set(xlim=(0, 100));
    plt.show()

In [ ]:
invest_profile_financials.max_equity.max()

#### The Financials will be imputed internally based on specific rules (tbd) and provided by the end of next week (latest: 04.03.2022)

## Deal Feature cleaning

In [ ]:
investor_deals = pd.read_excel('investor_deals.xlsx')

In [ ]:
investor_deals.head()

In [ ]:
investor_deals.columns.to_list()

In [ ]:
investor_deals = investor_deals[['investor_id', 'name', 
                                'deal_id', 'name.1',
                                'lost_reason_id', 'comment',
                                'target_name', 'target_description', 'target_revenue', 
                                'target_ebitda', 'target_ebit', 'target_country_id', 'name_en',
                                'target_region_id', 'target_zipcode', 'lead_prio', 
                                'type_id', 'name.2', 'is_lost', 'tx_type_id', 'name.3', 'ror_id', 'ror_name',
                                'region_id', 'name_en.1',
                                'sector_id.1', 'name_en.2', 'subsector_id', 'name_en.3']]

In [ ]:
investor_deals.head()

In [ ]:
deals_to_clean = investor_deals.copy()

In [ ]:
deals_to_clean.rename(columns={'name':'investor', 'name.1':'deal_name', 'name_en':'target_country_name',
                               'name.2':'deal_phase', 'name.3':'deal_type', 'name_en.1':'target_region_name',
                               'name_en.2':'sector', 'name_en.3':'subsector'}, inplace=True)

In [ ]:
deals_to_clean.type_id.value_counts()

In [ ]:
deals_to_clean['result'] = deals_to_clean['type_id'].apply(lambda x: 0 if x < 3 else 1)

In [ ]:
deals_to_clean['result'].value_counts().plot(kind="bar");

In [ ]:
deals_to_clean.drop(columns='lost_reason_id', inplace=True)

In [ ]:
deals_to_clean.columns

In [ ]:
deals_to_clean = deals_to_clean[['result', 'investor_id', 'investor', 'deal_id', 'deal_name',
                                 'type_id', 'deal_phase', 'is_lost', 'tx_type_id', 'deal_type', 'ror_id',
                                 'ror_name',  'comment',
                                 'target_name', 'target_description', 'target_revenue', 'target_ebitda',
                                 'target_ebit', 'target_country_id', 'target_country_name',
                                 'target_zipcode', 'region_id', 'target_region_name', 'sector_id.1', 'sector',
                                 'subsector_id', 'subsector']]

In [ ]:
deals_to_clean.sample()

#### Cleaning Tasks:

columns to One Hot Encode:
* deal_type
* ror_name
* target_country_name
* target_region_name
* sector

columns to scale:
* target_revenue
* target_ebitda
* target_ebit

columns to drop:
* type_id
* deal_phase
* tx_type_id
* ror_id
* target_name
* target_country_id
* region_id
* sector_id.1
* subsector_id

columns to drop and further investigate:
* comment
* target_description
* target_zipcode (<< future feature: distance target - investor)
* subsector


#### One Hot Encoding

In [ ]:
deals_to_clean.shape

In [ ]:
deals_to_clean[deals_to_clean.deal_type.isna()]['deal_name'].value_counts()

In [ ]:
deals_to_clean = deals_to_clean[deals_to_clean['deal_type'].notna()]

In [ ]:
deals_to_clean['deal_type'].isna().sum()

In [ ]:
# one hot encode deal types

one_hot_encoder = OneHotEncoder(sparse = False).fit(deals_to_clean[['deal_type']])

new_columns = list(one_hot_encoder.categories_[0])

deals_to_clean[new_columns] = one_hot_encoder.transform(deals_to_clean[['deal_type']])

deals_to_clean.head()

In [ ]:
deals_to_clean.shape

In [ ]:
deals_to_clean.ror_name.isna().sum()

In [ ]:
deals_to_clean.ror_name.replace(np.nan, 'NO_RESPONSE', inplace=True)

In [ ]:
# one hot encode ror_name

one_hot_encoder = OneHotEncoder(sparse = False).fit(deals_to_clean[['ror_name']])

new_columns = list(one_hot_encoder.categories_[0])

deals_to_clean[new_columns] = one_hot_encoder.transform(deals_to_clean[['ror_name']])

deals_to_clean.head()

In [ ]:
deals_to_clean.sector.isna().sum()

In [ ]:
# one hot encode sector

one_hot_encoder = OneHotEncoder(sparse = False).fit(deals_to_clean[['sector']])

new_columns = list(one_hot_encoder.categories_[0])

deals_to_clean[new_columns] = one_hot_encoder.transform(deals_to_clean[['sector']])

deals_to_clean.head()

consider: sectors_encoding_no_exclude_clean = sectors_encoding_no_exclude_clean.groupby('investor_id').max().reset_index()

In [ ]:
deals_to_clean.target_region_name.value_counts()

In [ ]:
deals_to_clean.target_region_name.isna().sum()

##### Question:
- What to do when no target_region is available? Is it meaningful to impute as "no_region_available"? 

#### Columns to drop

In [ ]:
deals_to_clean_1 = deals_to_clean.copy()

In [ ]:
cols_to_drop = ['type_id', 'deal_phase', 'tx_type_id', 'ror_id', 'target_name',
                'target_country_id', 'region_id', 'sector_id.1', 'subsector_id']

In [ ]:
deals_to_clean_1.drop(columns=cols_to_drop, inplace=True)
deals_to_clean_1.columns

In [ ]:
deals_to_clean_1.sample()

In [ ]:
deals_final = deals_to_clean_1.copy()

In [ ]:
deals_final.drop(columns=['investor', 'deal_name', 'deal_type', 'ror_name', 'comment',
                          'target_description', 'target_country_name', 'target_zipcode',
                          'target_region_name', 'sector', 'subsector'], inplace=True)

In [ ]:
deals_final.sample()

In [ ]:
deals_final.shape

In [ ]:
deals_final.target_revenue.hist(bins=1000)
plt.xlim(-10,200)

In [ ]:
deals_final.target_ebitda.hist(bins=100)
plt.xlim(-10,20)

In [ ]:
deals_final.target_ebit.hist(bins=100)
plt.xlim(-10,20)

In [ ]:
deals_final['target_ebit'] = np.where(deals_final['target_ebit'].isna(), deals_final['target_ebitda'], deals_final['target_ebit'])

In [ ]:
deals_final['target_ebit'].isna().sum()

In [ ]:
deals_final['target_ebitda'].isna().sum()

In [ ]:
deals_final['target_ebitda'] = np.where(deals_final['target_ebitda'].isna(), deals_final['target_ebit'], deals_final['target_ebitda'])

In [ ]:
deals_final['target_revenue'].isna().sum()

In [ ]:
deals_final = deals_final[deals_final['target_ebitda'].notna()]

In [ ]:
deals_final.target_revenue.isna().sum()

In [ ]:
deals_final = deals_final[deals_final['target_revenue'].notna()]

In [ ]:
deals_final = deals_final[deals_final['target_ebitda'] != 0]

In [ ]:
deals_final.head()

In [ ]:
deals_final.shape

In [ ]:
deals_final_2 = deals_final.copy()

In [ ]:
deals_final_2 = deals_final_2.groupby('deal_id').max().reset_index()

In [ ]:
deals_final_2.investor_id.nunique()

In [ ]:
deals_final_2

In [ ]:
deals_final_2.boxplot('target_ebit')
plt.ylim(0,20)

In [ ]:
from sklearn.preprocessing import RobustScaler

r_scaler = RobustScaler() # Instanciate Robust Scaler

r_scaler.fit(deals_final_2[['target_revenue']]) # Fit scaler to feature

deals_final_2['target_revenue'] = r_scaler.transform(deals_final_2[['target_revenue']]) #Scale

deals_final_2.head()

In [ ]:
from sklearn.preprocessing import RobustScaler

r_scaler = RobustScaler() # Instanciate Robust Scaler

r_scaler.fit(deals_final_2[['target_ebitda']]) # Fit scaler to feature

deals_final_2['target_ebitda'] = r_scaler.transform(deals_final_2[['target_ebitda']]) #Scale

deals_final_2.head()

In [ ]:
from sklearn.preprocessing import RobustScaler

r_scaler = RobustScaler() # Instanciate Robust Scaler

r_scaler.fit(deals_final_2[['target_ebit']]) # Fit scaler to feature

deals_final_2['target_ebit'] = r_scaler.transform(deals_final_2[['target_ebit']]) #Scale

deals_final_2.head()

In [ ]:
deals_final_2.shape

In [ ]:
deals_final_2.investor_id.nunique()

##### final df to merge: deals_final_2